In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
spark = SparkSession.builder.appName("Ngram_buzzwords").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [2]:
df = spark.read.csv('../data/eng-gb-all-only-letters.csv', sep="\t", header=False)

In [3]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [4]:
#df.count()

In [5]:
rdd = df.rdd.map(list)

In [6]:
def splitAndMakeNgramKey(row):
    elements = row[0].split(",")
    return(elements[0], (elements[1:len(elements)]))

rdd = rdd.map(splitAndMakeNgramKey)

In [7]:
rdd.take(10)

[('A', ['1541', '8', '7', '1']),
 ('A', ['1547', '112', '77', '1']),
 ('A', ['1551', '40', '30', '1']),
 ('A', ['1562', '378', '196', '1']),
 ('A', ['1576', '41', '26', '1']),
 ('A', ['1582', '17', '17', '1']),
 ('A', ['1584', '15', '13', '1']),
 ('A', ['1588', '51', '38', '1']),
 ('A', ['1590', '43', '26', '1']),
 ('A', ['1591', '25', '20', '1'])]

In [36]:
rdd2 = sc.parallelize(rdd.take(1000000))

In [37]:
def groupData(a,b):
    if isinstance(a[0], (list,)):
        return a+[b]
    else:
        return [a]+[b]

rdd3 = rdd2.reduceByKey(groupData)

In [38]:
rdd3.take(10)

[('ARRS',
  [['1797', '1', '1', '1'],
   ['1845', '2', '2', '2'],
   ['1869', '1', '1', '1'],
   ['1885', '1', '1', '1'],
   ['1887', '3', '3', '3'],
   ['1889', '1', '1', '1'],
   ['1900', '1', '1', '1'],
   ['1928', '1', '1', '1'],
   ['1977', '2', '2', '2'],
   ['1978', '2', '2', '2'],
   ['1979', '1', '1', '1'],
   ['1980', '10', '10', '3'],
   ['1981', '3', '3', '3'],
   ['1987', '7', '5', '2'],
   ['1988', '19', '15', '5'],
   ['1991', '29', '9', '3'],
   ['1994', '4', '4', '1'],
   ['1997', '2', '2', '1'],
   ['1999', '2', '1', '1'],
   ['2001', '7', '6', '6'],
   ['2002', '3', '2', '1'],
   ['2003', '13', '10', '4'],
   ['2005', '45', '15', '2']]),
 ('AUCTORE',
  [['1712', '1', '1', '1'],
   ['1741', '1', '1', '1'],
   ['1748', '2', '2', '2'],
   ['1749', '1', '1', '1'],
   ['1801', '3', '3', '3'],
   ['1804', '4', '4', '4'],
   ['1806', '1', '1', '1'],
   ['1807', '2', '2', '2'],
   ['1808', '4', '4', '2'],
   ['1810', '4', '4', '4'],
   ['1811', '1', '1', '1'],
   ['1816', '1

In [41]:
rdd3.coalesce(1).saveAsTextFile("test-data-for-johan.txt")

12332